In [1]:
#Importing all the packages
import pandas as pd
import matplotlib.pyplot as plt
plt.style.use('seaborn')
%matplotlib inline
from cobra.io import read_sbml_model
from cobra.io import write_sbml_model

In [2]:
#Importing the model "iCW773"with changed names --> iCW773_test7.xml
model = read_sbml_model('iCW773_test7.xml') #for test file e.g. trp- was replaced by trp_ to enable calling of the metabolite



No objective coefficients in model. Unclear what should be optimized


Adding metabolites to the model

In [3]:
from cobra import Reaction, Metabolite
tryptamine = Metabolite(id = 'tryp_c', compartment='c', name='tryptamine', formula='C10H12N2')
hydroxytryptamine = Metabolite(id='_4htryp_c', compartment='c', name='4-Hydroxytryptamine', formula='C10H12N2O')
norbaeocystin = Metabolite(id='nbc_c' , compartment='c', name='Norbaeocystin', formula='C10H13N2O4P')
psilocybin = Metabolite(id='psi_c', compartment='c', name='Psilocybin', formula='C12H17N2O4P') #This needs later also an _e version
#S_adenosyl_L_methionine = Metabolite(id='SAM', compartment='c', name='S-adenosyl-L-Methionine', formula='C15H22N6O5S')
psilocin = Metabolite(id='psii_c', compartment='c', name='psilocin', formula='C12H16N2O')
baeocystin = Metabolite(id='baecyst_c', compartment='c', name='Baeocystin', formula='C11H15N2O4P')

## Defining the reactions

In [4]:
L_tryptophan_decarboxylase = Reaction('CrTdc')
tryptamine_4_monooxygenase = Reaction('psiH')
_4_hydroxytryptamine_kinase = Reaction('psiK')
psilocybin_synthase = Reaction('psiM')
psilocybin_synthase2 = Reaction('psiM2')
pho_or_spon = Reaction('pho') # reaction is either spontaneous or carried out by phosphatases
_4_hydroxytryptamine_kinase2 = Reaction('psiK2')

## Adding the metabolites to the reactions


In [5]:
L_tryptophan_decarboxylase.add_metabolites(({model.metabolites.trp_L_c: -1,
                              tryptamine: 1, 
                              model.metabolites.co2_c: 1, 
                            }))

In [6]:
tryptamine_4_monooxygenase.add_metabolites({tryptamine: -1,
                              model.metabolites.nadph_c: -1, # this is from the CP450 reductase reaction WE THINK?              
                              model.metabolites.o2_c: -1,
                              model.metabolites.h_c: -2,
                              hydroxytryptamine: 1,
                              model.metabolites.nadp_c:1,
                              model.metabolites.h2o_c: 1,              
                             })

#####Check this reaction again regardin the amount of protons!!!!

In [7]:
_4_hydroxytryptamine_kinase.add_metabolites({hydroxytryptamine: -1,
                              model.metabolites.atp_c: -1,
                              norbaeocystin: 1, 
                              model.metabolites.adp_c: 1,
                             })

In [8]:
psilocybin_synthase.add_metabolites(({norbaeocystin: -1,
                             model.metabolites.amet_c: -1, 
                             baeocystin: 1, 
                             model.metabolites.ahcys_c: 1,
                             }))

In [9]:
psilocybin_synthase2.add_metabolites(({baeocystin: -1,
                             model.metabolites.amet_c: -1, 
                             psilocybin: 1, 
                             model.metabolites.ahcys_c: 1,
                             }))

In [10]:
pho_or_spon.add_metabolites(({psilocybin: -1,
                             model.metabolites.h_c: -1,
                             psilocin: 1,
                             model.metabolites.pi_c: 1,
                             }))

In [11]:
_4_hydroxytryptamine_kinase2.add_metabolites(({psilocin: -1,
                             model.metabolites.atp_c: -1, 
                             psilocybin: 1, 
                             model.metabolites.adp_c: 1, 
                             }))

## Adding the reactions to the model

In [12]:
model.add_reactions([L_tryptophan_decarboxylase]) #
model.add_reactions([tryptamine_4_monooxygenase]) #
model.add_reactions([_4_hydroxytryptamine_kinase]) #
model.add_reactions([_4_hydroxytryptamine_kinase2]) #
model.add_reactions([psilocybin_synthase]) #
model.add_reactions([psilocybin_synthase2]) #
model.add_reactions([pho_or_spon]) #

In [13]:
model.add_boundary(model.metabolites.tryp_c, type='demand')
model.add_boundary(model.metabolites._4htryp_c, type='demand')
model.add_boundary(model.metabolites.nbc_c, type='demand')
model.add_boundary(model.metabolites.psi_c, type='demand')
model.add_boundary(model.metabolites.amet_c, type='demand')
model.add_boundary(model.metabolites.ahcys_c, type='demand')
model.add_boundary(model.metabolites.psii_c, type='demand');

In [14]:
model.metabolites.tryp_c

Metabolite identifier,tryp_c
Name,tryptamine
Memory address,0x02197fbc8460
Formula,C10H12N2
Compartment,c
In 3 reaction(s),"DM_tryp_c, CrTdc, psiH"


In [16]:
write_sbml_model(model, "iCW773_test7_psilocybin.xml")